In [1]:
import itertools as it
import numpy as np
import fenics as fc
import tools

In [2]:
# Imports for plotting
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
# Has side effects allowing for 3D plots
from mpl_toolkits.mplot3d import Axes3D

In [3]:
### Define Mesh sizes
t, T = 0.0, 50.0  # start and end times
a, b = 0, 40      # start and end spacial positions
fineness = tools.Object(t=0.01, x=0.01)

nt = round(T / fineness.t)          # number of time steps
nx = round((b - a) / fineness.x)         # number of space steps
dt = fineness.t

### Define periodic boundary
class PeriodicBoundary(fc.SubDomain):

    def inside(self, x, on_boundary):
        return bool(x[0] < fc.DOLFIN_EPS and x[0] > -fc.DOLFIN_EPS and on_boundary)

    # Map right boundary to left boundary
    def map(self, x, y):
        y[0] = x[0] - (b - a)

### Create mesh and define function spaces
mesh = fc.IntervalMesh(nx, a, b)
F_ele = fc.FiniteElement("CG", mesh.ufl_cell(), 1)
V = fc.FunctionSpace(mesh, F_ele, 
                     constrained_domain=PeriodicBoundary())
W = fc.FunctionSpace(mesh, fc.MixedElement([F_ele, F_ele]), 
                     constrained_domain=PeriodicBoundary())

In [4]:
### Define and deduce initial values
u0_uninterpolated = fc.Expression('2/cosh(x[0] - 403.0/15.0) + 5 / cosh(x[0] - 203.0/15.0)', degree=1)
#u0_uninterpolated = fc.Expression('0.2 * exp(-fabs(x[0] - 10)) + exp(-fabs(x[0] - 5))', degree=1)
u0 = fc.interpolate(u0_uninterpolated, V)

# Find initial value m0 for m
q = fc.TestFunction(V)
m = fc.TrialFunction(V)

dx = fc.dx
am = q * m * dx
Lm = (q * u0 + q.dx(0) * u0.dx(0)) * dx
      
m0 = fc.Function(V)
fc.solve(am == Lm, m0)

In [5]:
### Storing results
# We begin by storing the intial condition.
uvals = [u0.compute_vertex_values(mesh)]

In [6]:
### State variational problem
p, q = fc.TestFunctions(W)
w = fc.Function(W)  # Function to solve for
m, u = fc.split(w)
# Relabel i.e. initialise m_prev, u_prev as m0, u0.
m_prev, u_prev = m0, u0
m_mid = 0.5 * (m + m_prev)
u_mid = 0.5 * (u + u_prev)
F = (
    (q * u + q.dx(0) * u.dx(0) - q * m) * dx +                                          # q part
    (p * (m - m_prev) + dt * (p * m_mid * u_mid.dx(0) - p.dx(0) * m_mid * u_mid)) * dx  # p part
    )
J = fc.derivative(F, w)
problem = fc.NonlinearVariationalProblem(F, w, J=J)
solver = fc.NonlinearVariationalSolver(problem)

In [7]:
### Time step through the problem
for n in range(nt):
#     if n % 10 == 0:
#         E = assemble((u_prev * u_prev + u_prev.dx(0) * u_prev.dx(0)) * dx)
#         print("time {:0>4} energy {}".format(t, E))  # Energy should remain constant
    t += dt
    solver.solve()
    # For some reason m_prev.assign(m) fails unless a deepcopy is made here
    m, u = w.split(deepcopy=True)
    uvals.append(u.compute_vertex_values(mesh))  # Save result
    m_prev.assign(m)  # Update for next loop
    u_prev.assign(u)  #

In [8]:
uvals = np.array(uvals)
xvals = mesh.coordinates()[:, 0]
tvals = np.linspace(0, T, nt + 1)

In [9]:
# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# # X, Y = np.meshgrid(xvals, tvals)
# # ax.plot_trisurf(X.flatten(), Y.flatten(), uvals.flatten())
# ax.plot_surface(*np.meshgrid(xvals, tvals), uvals)
# ax.set_xlabel('x')
# ax.set_ylabel('t')

In [12]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
#ax.axis('off')
step = round(1 / fineness.t)
base = np.zeros(uvals.shape[1])
for i in range(0, len(uvals) + 1, step):
    zorder = len(uvals) - round(i / step)
    offset = 0.01 * i
    ax.fill_between(xvals, offset + uvals[i], offset, color='white', zorder=zorder)
    ax.plot(xvals, offset + uvals[i], color='black', zorder=zorder)

FigureCanvasNbAgg()